In [1]:
import copy
import itertools
from itertools import *

# Taken from: https://docs.python.org/3/library/itertools.html
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def debug(s, d):
    if d:
        print(s)

In [2]:
# Find and return a valid assignment by brute force, or None if there isn't one.

def brute_force(lands, types):
    if len(lands) > len(types):
        return None
    
    if [l for l in lands if len(l) == 0]:
        return None

    # If not all types covered between all of the lands, no valid assignment exists.
    if set(itertools.chain(*lands)) != set(types):
        return None    
    
    # Try all possible combinations of assignments of a type to a land, considering only assignments
    # where the land has that type
    initial_assignment = None
    for index_combo in itertools.product(*lands):
        # If we find an assignment where each land is assigned a unique type, success.
        if len(set(index_combo)) == len(lands):
            initial_assignment = index_combo
            break
    
    if initial_assignment is None:
        return None

    # If we got this far, then a valid assignment exists because we've already found one type
    # for each land, with no two lands sharing a type. For any remaining types, just assign them
    # to any land which has its type.    
    assignment = []
    unassigned_types = types.copy()
    for t in initial_assignment:
        unassigned_types.remove(t)
        assignment.append([t])

    while unassigned_types:
        t = unassigned_types.pop()
        for i in range(0, len(lands)):
            if t in lands[i]:
                assignment[i].append(t)
                break

    return assignment

In [3]:
# Find and return a valid assignment. General approach is to assign a type to a land one at a time,
# choosing an assignment that removes the least options.

def find_assignment(lands, types, d=False):
    if len(lands) > len(types):
        return None

    # If not all types covered between all of the lands, no valid assignment exists.
    if set(itertools.chain(*lands)) != set(types):
        debug('not all types covered', d)
        return None
    
    lands_original = lands
    lands, types = copy.deepcopy(lands), copy.deepcopy(types)
    
    assignment = [[] for _ in lands]
    num_assigned = 0
    
    # We already determined that all land types are covered between the lands. Now, we just need
    # to find a type to assign to each land, with no two lands sharing an assigned type.
    while num_assigned < len(lands):
        debug('lands: {} | types: {} | assignment: {}'.format(lands, types, assignment), d)

        # Find the "least flexible" type, i.e. the type with the smallest number of valid lands
        # which has at least one valid land.
        type_chosen, min_value = None, None
        for t in types:
            num_valid_lands = len([l for l in lands if t in l])
            if num_valid_lands > 0 and (min_value is None or num_valid_lands < min_value):
                min_value, type_chosen = num_valid_lands, t
        debug('type chosen: ' + str(type_chosen), d)

        # If there is no type with at least one valid land, no valid assignment exists.
        if min_value is None:
            debug('Min value is None', d)
            return None

        # Now we have chosen the type to assign, we need to choose the land. We also want to choose
        # the "least flexible" land, i.e. the one that has the least other types.
        valid_land_indices = [i for i in range(0, len(lands)) if type_chosen in lands[i]]
        debug('valid land indices: ' + str(valid_land_indices), d)

        land_index, min_value = None, None
        for i in valid_land_indices:
            num_options = len(lands[i])
            if min_value is None or num_options < min_value:
                land_index, min_value = i, num_options
        debug('land index chosen: ' + str(land_index), d)

        # Now that we've picked the assignment, update assignment, remove the type, and remove
        # the assigned types from all lands.
        assignment[land_index].append(type_chosen)
        lands[land_index].clear()
        types.remove(type_chosen)
        for l in lands:
            if type_chosen in l:
                l.remove(type_chosen)
        num_assigned += 1

    # If we got this far, then a valid assignment exists because we've already found one type
    # for each land, with no two lands sharing a type. For any remaining types, just assign them
    # to any land which has its type.
    while types:
        t = types.pop()
        for i in range(0, len(lands_original)):
            if t in lands_original[i]:
                assignment[i].append(t)
                break
    
    return assignment

In [4]:
# For a given number of types, goes through all possible combinations of
# number of lands and types for those lands, and compares the brute force result
# with the algorithm result. Raises exception if there's a differing result.

def check_algorithm(num_types, d=True):
    types = list(range(1, num_types + 1))
    
    for num_lands in range(1, num_types + 1):
        p = powerset(types)
        for combination in itertools.combinations_with_replacement(p, r=num_lands):
            lands = [list(tup) for tup in combination]
            brute_force_result = brute_force(lands, types)
            algo_result = find_assignment(lands, types)
            
            debug_message = 'lands: {} | brute force: {} | algorithm: {}'.format(lands, brute_force_result, algo_result)
            
            debug(debug_message, d)
            if (brute_force_result is None) != (algo_result is None):
                raise Exception('Differing results: ' + debug_message)
    print('Success!')


In [5]:
check_algorithm(3)

lands: [[]] | brute force: None | algorithm: None
lands: [[1]] | brute force: None | algorithm: None
lands: [[2]] | brute force: None | algorithm: None
lands: [[3]] | brute force: None | algorithm: None
lands: [[1, 2]] | brute force: None | algorithm: None
lands: [[1, 3]] | brute force: None | algorithm: None
lands: [[2, 3]] | brute force: None | algorithm: None
lands: [[1, 2, 3]] | brute force: [[1, 3, 2]] | algorithm: [[1, 3, 2]]
lands: [[], []] | brute force: None | algorithm: None
lands: [[], [1]] | brute force: None | algorithm: None
lands: [[], [2]] | brute force: None | algorithm: None
lands: [[], [3]] | brute force: None | algorithm: None
lands: [[], [1, 2]] | brute force: None | algorithm: None
lands: [[], [1, 3]] | brute force: None | algorithm: None
lands: [[], [2, 3]] | brute force: None | algorithm: None
lands: [[], [1, 2, 3]] | brute force: None | algorithm: None
lands: [[1], [1]] | brute force: None | algorithm: None
lands: [[1], [2]] | brute force: None | algorithm: Non

In [6]:
check_algorithm(4, d=False)

Success!


In [7]:
check_algorithm(5, d=False)

Exception: Differing results: lands: [[2], [1, 3], [2, 3], [1, 4, 5], [1, 2, 4, 5]] | brute force: [[2], [1], [3], [4], [5]] | algorithm: None

In [ ]:
# Just used for debugging, not important

result = find_assignment([[1, 2], [1, 2], [2, 3], [4, 5]], [1, 2, 3, 4, 5], d=True)
print(result)

In [ ]:
# Just used for debugging, not important

result = find_assignment([[1, 2], [1, 2], [2, 3], [2, 3], [4, 5]], [1, 2, 3, 4, 5], d=True)
print(result)